# Titanicデータを用いたデータ分析<br>
## Predictive Modeling編
alleeksテックブログ機械学習編のコードです。kaggleのnotebook状で実行可能なコードになっています。<br>
Feature Engineering編の続きからになっていますので、もし手元で確認される場合は、Feature Engineering編のコードを実行した後に、続けてください。

In [ ]:
#以降使用するライブラリをimportする
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier 
import lightgbm as lgm
import xgboost as xg
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.metrics import confusion_matrix 

In [ ]:
#トレーニングデータをtrainとvalidに分割する
train, valid = train_test_split(train_df, test_size=0.3, random_state=0, stratify=train_df["Survived"])
train_X = train[train.columns[1:]]
train_Y = train[train.columns[:1]]
valid_X = valid[valid.columns[1:]]
valid_Y = valid[valid.columns[:1]]
#XGBoostは引数でvalidationデータのサイズを指定すればvalidationできるので、
#そのために分割していないデータを作成しておく
X = train_df[train_df.columns[1:]]
Y = train_df[train_df.columns[:1]]

In [ ]:
#ロジスティック回帰の学習・validationデータで精度を見積もる
LRmodel = LogisticRegression()
LRmodel.fit(train_X,train_Y)
prediction1=LRmodel.predict(valid_X)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(prediction1,valid_Y))

In [ ]:
#決定木の学習・validationデータで精度を見積もる
DTCmodel=DecisionTreeClassifier()
DTCmodel.fit(train_X,train_Y)
prediction2=DTCmodel.predict(valid_X)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction2,valid_Y))

In [ ]:
#ランダムフォレストで学習・validationデータで精度を見積もる
RFCmodel=RandomForestClassifier(n_estimators=100)
RFCmodel.fit(train_X,train_Y)
prediction3=RFCmodel.predict(valid_X)
print('The accuracy of the Random Forests is',metrics.accuracy_score(prediction3,valid_Y))

In [ ]:
#LightGBMで学習
LGBMmodel=lgm.LGBMClassifier(objective="binary")
LGBMmodel.fit(train_X,train_Y, eval_set=[(valid_X, valid_Y)], early_stopping_rounds=20, verbose=10)

In [ ]:
#LightGBMの学習結果の精度をvalidationデータで見積もる
prediction4=LGBMmodel.predict(valid_X)
print('The accuracy of the Random Forests is',metrics.accuracy_score(prediction4,valid_Y))

In [ ]:
#XGBoostで学習・validationデータで精度を見積もる
XGmodel=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
predict5=cross_val_score(XGmodel,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for XGBoost is:',predict5.mean())

In [ ]:
#テストデータの推定を行うための準備を行う
test_df.head()

In [ ]:
#Ageカラムを補間するためにhonorificsカラムを作成する
test_df['honorifics'] = test_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
#テストデータに含まれる敬称を確認する。トレーニングデータと比較する
test_df.Name.str.extract("([A-Za-z]+)\.", expand=False).value_counts()

In [ ]:
#トレーニングデータには含まれない敬称の"Dona"を持つデータを確認する
test_df[test_df["honorifics"] == "Dona"]

In [ ]:
#敬称を元にAgeカラムを補間する。
#補間するための値はトレーニングデータで用いた値と同じものを用いる
test_df.loc[(test_df.Age.isnull())&(test_df.honorifics=="Mr"), "Age"]=32
test_df.loc[(test_df.Age.isnull())&(test_df.honorifics=="Miss"), "Age"]=22
test_df.loc[(test_df.Age.isnull())&(test_df.honorifics=="Mrs"), "Age"]=36
test_df.loc[(test_df.Age.isnull())&(test_df.honorifics=="Master"), "Age"]=5
test_df.loc[(test_df.Age.isnull())&(test_df.honorifics=="Rev"), "Age"]=43
test_df.loc[(test_df.Age.isnull())&(test_df.honorifics=="Col"), "Age"]=58
test_df.loc[(test_df.Age.isnull())&(test_df.honorifics=="Ms"), "Age"]=28
test_df.loc[(test_df.Age.isnull())&(test_df.honorifics=="Dr"), "Age"]=42

In [ ]:
#Fareカラムが欠損しているデータを確認する
test_df.loc[test_df.Fare.isnull()]

In [ ]:
#テストデータFareカラムの欠損しているデータと似た情報を持つデータをトレーニングデータから抜き出す
train2_df = pd.read_csv('../input/titanic/train.csv')
train_df[(train2_df.Age>= 55)&(train2_df.Age<=65)&(train2_df.Sex=="male")&(train2_df.Embarked=="S")]

In [ ]:
#抜き出したデータのFareの平均値を計算する。
train_df[(train2_df.Age>= 55)&(train2_df.Age<=65)&(train2_df.Sex=="male")&(train2_df.Embarked=="S")].Fare.mean()

In [ ]:
#上記で計算した平均値でテストデータを補間する
test_df.loc[(test_df.Fare.isnull()), "Fare"]=38

In [ ]:
#テストデータでもFamilyカラムとAloneカラムを作成する
test_df["Family"] = 0
test_df["Family"] = test_df["Parch"] + test_df["SibSp"]
test_df["Alone"] = 0
test_df.loc[test_df.Family==0, "Alone"] = 1

In [ ]:
#Sexカラム・Embarkedカラムを文字列型から数値型に変換する
test_df["Sex"].replace(["male", "female"], [0, 1], inplace=True)
test_df["Embarked"].replace(["S", "C", "Q"], [0, 1, 2], inplace=True)

In [ ]:
#不要なカラムを削除する
test_df.drop(["PassengerId", "Name", "Ticket", "honorifics", "Cabin"], axis=1, inplace=True)

In [ ]:
#学習に用いたトレーニングデータと同じカラム・値になっているか確認する
test_df.head()

In [ ]:
#validation結果が一番良かったランダムフォレストのモデルを用いてテストデータのSurvivedを予測
prediction = RFCmodel.predict(test_df)

In [ ]:
#予測結果を整理して、csvファイルにする（kaggle特有の処理）
submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission["Survived"] = prediction
submission.to_csv('submission.csv', index=False)